In [14]:
import http.server
import socketserver
import json
from urllib.parse import urlparse, parse_qs
from pyngrok import ngrok, conf
from transformers import pipeline
import os
import threading

# Set your ngrok authtoken
NGROK_AUTH_TOKEN = "2geUw5OKJ1IarcHEvhg0Oh3wgkw_61xjuKWRFDKAP9ng4A3oc"  # Replace with your ngrok authtoken
conf.get_default().auth_token = NGROK_AUTH_TOKEN


# Load the NLP model for text classification
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

# Define candidate labels for classification
candidate_labels = ["greeting", "covid_safety", "opening_hours", "table_availability", "reserve_table", "menu", "food_order"]

# Define the chatbot responses based on intents
intent_responses = {
    'greeting': 'Hello! How can I help you?',
    'covid_safety': 'We follow all WHO recommended protocols to ensure your safety.',
    'opening_hours': 'We are open 10am-12am Monday to Friday!',
    'table_availability': 'There are 48 tables available at the moment.',
    'reserve_table': 'Your table has been booked successfully. Please show this Booking ID at the counter: 25',
    'menu': 'Our menu includes a variety of dishes such as pasta, pizza, salads, and desserts. What would you like to have?',
    'food_order': 'Your order for {item} has been placed. Anything else you would like to order?',
    'unknown': 'Sorry, I didn\'t understand that. Could you please rephrase?'
}

# List of specific food items
food_items = ["pasta", "pizza", "salad", "dessert"]

def get_intent(user_message):
    result = classifier(user_message, candidate_labels)
    intent = result['labels'][0]
    print(f"Detected intent: {intent}")
    return intent if intent in intent_responses else 'unknown'

def generate_response(intent, user_message):
    if intent == 'food_order':
        for item in food_items:
            if item in user_message.lower():
                return intent_responses[intent].format(item=item)
        return intent_responses['menu']
    else:
        return intent_responses[intent]

class MyHttpRequestHandler(http.server.SimpleHTTPRequestHandler):
    def do_POST(self):
        if self.path == '/api/chat':
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length)
            user_message = json.loads(post_data)['message']

            intent = get_intent(user_message)
            response = generate_response(intent, user_message)

            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps({'response': response}).encode('utf-8'))
        else:
            self.send_error(404, 'File Not Found: %s' % self.path)

# Find an available port
def find_free_port():
    with socketserver.TCPServer(("localhost", 0), None) as s:
        return s.server_address[1]

PORT = find_free_port()
Handler = MyHttpRequestHandler
httpd = socketserver.TCPServer(("", PORT), Handler)

# Start ngrok tunnel
public_url = ngrok.connect(PORT)
print(f" * ngrok tunnel {public_url} -> http://127.0.0.1:{PORT}")

# Serve files
!mkdir -p /content/web
%cd /content/web

# Create index.html
with open('index.html', 'w') as f:
    f.write('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Bestos Chatbot</title>
        <link rel="stylesheet" href="styles.css">
    </head>
    <body>
        <div class="container">
            <div class="chat-header">
                <h1>Welcome to Bestos</h1>
            </div>
            <div class="chat-box" id="chat-box">
                <!-- Chat messages will be appended here -->
            </div>
            <input type="text" id="user-input" placeholder="Ask me anything..." onkeydown="if(event.key === 'Enter') sendMessage()">
        </div>
        <script src="script.js"></script>
    </body>
    </html>
    ''')

# Create styles.css
with open('styles.css', 'w') as f:
    f.write('''
    body {
        font-family: 'Arial', sans-serif;
        background-color: #f4f4f9;
        margin: 0;
        padding: 0;
        display: flex;
        justify-content: center;
        align-items: center;
        height: 100vh;
        background: linear-gradient(to right, #e0c3fc 0%, #8ec5fc 100%);
    }

    .container {
        width: 450px;
        max-width: 100%;
        background-color: white;
        border-radius: 10px;
        box-shadow: 0 10px 30px rgba(0, 0, 0, 0.1);
        overflow: hidden;
        display: flex;
        flex-direction: column;
        align-items: center;
    }

    .chat-header {
        background-color: #6c63ff;
        width: 100%;
        padding: 20px;
        text-align: center;
        color: white;
        font-size: 24px;
        font-weight: bold;
    }

    .chat-box {
        flex: 1;
        width: 100%;
        padding: 20px;
        overflow-y: auto;
        border-bottom: 1px solid #ddd;
        background-color: #f7f7f7;
    }

    .chat-box .message {
        margin: 10px 0;
        padding: 10px;
        border-radius: 5px;
        max-width: 80%;
        word-wrap: break-word;
    }

    .message.bot {
        background-color: #e1e1e1;
        align-self: flex-start;
        color: #333;
    }

    .message.user {
        background-color: #6c63ff;
        align-self: flex-end;
        color: white;
    }

    input[type="text"] {
        width: calc(100% - 40px);
        padding: 15px;
        border: none;
        border-top: 1px solid #ddd;
        font-size: 16px;
        outline: none;
    }
    ''')

# Create script.js
with open('script.js', 'w') as f:
    f.write('''
    document.addEventListener('DOMContentLoaded', (event) => {
        document.getElementById('user-input').focus();
    });

    function sendMessage() {
        const inputBox = document.getElementById('user-input');
        const message = inputBox.value;
        if (message.trim() === '') return;

        appendMessage('user', message);
        inputBox.value = '';

        // Send the message to the backend
        fetch('/api/chat', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            },
            body: JSON.stringify({ message })
        })
        .then(response => response.json())
        .then(data => {
            appendMessage('bot', data.response);
        });
    }

    function appendMessage(sender, message) {
        const chatBox = document.getElementById('chat-box');
        const messageDiv = document.createElement('div');
        messageDiv.classList.add('message', sender);
        messageDiv.textContent = message;
        chatBox.appendChild(messageDiv);
        chatBox.scrollTop = chatBox.scrollHeight;
    }
    ''')

# Start the server in a new process
def run_server():
    httpd.serve_forever()

server_thread = threading.Thread(target=run_server)
server_thread.daemon = True
server_thread.start()


 * ngrok tunnel NgrokTunnel: "https://be72-34-28-14-6.ngrok-free.app" -> "http://localhost:45501" -> http://127.0.0.1:45501
/content/web
